In [ ]:
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
import subprocess
import csv
import os
import pandas as pd
import seaborn as sns
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neural_network import MLPClassifier


In [ ]:
# lista_de_arquivos = []
# for nome_do_arquivo in os.listdir('imagens/train/T'):
#         # Verifica se é um arquivo (e não um diretório)
#         if os.path.isfile(os.path.join('imagens/train/T', nome_do_arquivo)):
#             lista_de_arquivos.append(nome_do_arquivo)
# lista_de_arquivos.sort()

In [ ]:
# indice_inicial = 413
# for i in range(indice_inicial, len(lista_de_arquivos)):
#     nome_atual = f'./imagens/train/T/{lista_de_arquivos[i]}'
#     novo_indice = i + 1
#     nome_novo = f'./imagens/train/T/{novo_indice}.png'
#     os.rename(nome_atual, nome_novo)

In [ ]:
label = ["A","B","C","D","E","F","G","I","L","M","N","O","P","Q","R","S","T","U","V","W"]

landmarks = ["WRIST",
    "THUMB_CMC",
    "THUMB_MCP",
    "THUMB_IP",
    "THUMB_TIP",
    "INDEX_FINGER_MCP",
    "INDEX_FINGER_PIP",
    "INDEX_FINGER_DIP",
    "INDEX_FINGER_TIP",
    "MIDDLE_FINGER_MCP",
    "MIDDLE_FINGER_PIP",
    "MIDDLE_FINGER_DIP",
    "MIDDLE_FINGER_TIP",
    "RING_FINGER_MCP",
    "RING_FINGER_PIP",
    "RING_FINGER_DIP",
    "RING_FINGER_TIP",
    "PINKY_MCP",
    "PINKY_PIP",
    "PINKY_DIP",
    "PINKY_TIP"]

In [ ]:
# Obtendo o numero de arquivos de imagem de cada letra
num_files = [] # cada elemento da lista é o numero de arquivos de imagem das letras em ordem alfabética
for i,reps in enumerate(label):
    # Caminho do diretório
    directory = f'./imagens/test/{reps}'
    # Executar comando bash para contar o número de arquivos
    count_subprocess_result = subprocess.run(f'find {directory} -type f | wc -l', stdout=subprocess.PIPE, shell=True, text=True)
    output = count_subprocess_result.stdout.strip()
    num_files.append(int(output))

In [ ]:
sum(num_files)

In [ ]:
  # Initialize the MediaPipe Hands object
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)


In [ ]:
csv_filename = 'test_centro_geometrico.csv'
header = [f'Landmark{i}_{axis}' for i in range(0, 21) for axis in ['x', 'y', 'z']]
header.append('label')


In [ ]:
!mkdir imagens_com_problemas_teste

In [ ]:
!rm imagens_com_problemas_teste/* # Limpando o diretorio de imagens com problemas
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header) # Inserindo os nomes dos atributos
    for i, reps in enumerate(label): 
        for j in range(1,num_files[i]+1):
            newRow = []
            if(reps == 'F' and j > 300):
                k = j + 100
            else:
                k = j

            caminho = "imagens/test/" + reps + "/" + str(k) + ".png"
            image = cv2.imread(caminho)
            #image_bgr_resized = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
            # Convert the image to RGB format
            try:
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Process the image with MediaPipe Hands
                results = hands.process(image_rgb)
                # Draw hand landmarks on the image
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_world_landmarks:
                        # Print the coordinates of the landmarks
                        for landmark in hand_landmarks.landmark:
                            newRow.append(landmark.x)
                            newRow.append(landmark.y)
                            newRow.append(landmark.z)
                    newRow.append(reps)
                    writer.writerow(newRow)
                else:
                    cv2.imwrite("imagens_com_problemas_teste/imagem" + reps + "00" + str(k) + ".png", image)
            except Exception as e:
                print(caminho)

In [ ]:
# Draw hand landmarks on the image
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        # Print the coordinates of the landmarks
        for landmark in hand_landmarks.landmark:
            print(f"Landmark coordinates: ({landmark.x}, {landmark.y}, {landmark.z})")

# Display the image with hand landmarks
plt.imshow(image_rgb)

In [ ]:
df_train = pd.read_csv('treino_centro_geometrico.csv')
df_train_sample = df_train.sample(n=10000)
plt.figure(figsize=(30, 30))
sns.scatterplot(x='Landmark8_x', y='Landmark8_y', hue='label', style='label', data=df_train_sample[(df_train_sample['label'] == 'D') | (df_train_sample['label'] == 'Q') | (df_train_sample['label'] == 'A') ], s=300)


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100, 100), random_state=1, max_iter = 10000)


In [ ]:
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
df_test = pd.read_csv('test_centro_geometrico.csv')

In [ ]:
X_test = df_test.drop(columns=['label'])
y_test = df_test['label']

In [ ]:
results = clf.predict(X_test)

In [ ]:
df_results_compare = pd.DataFrame(results == y_test)
df_results_compare['label'].value_counts()
    

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Parâmetros
image_size = (64, 64)  
batch_size = 32
epochs = 20

# Caminhos para os dados
train_data_dir = 'imagens/train/'
test_data_dir = 'imagens/test/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  

# Compilação do modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

In [ ]:
# Treinamento do modelo
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    verbose=1
)

In [ ]:
# Salvar o modelo após o treinamento
model.save('modelo_libras.h5')

In [ ]:
import matplotlib.pyplot as plt

# Plotando a acurácia
plt.plot(history.history['accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.show()

# Plotando a perda
plt.plot(history.history['loss'])
plt.title('Perda do modelo')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.show()


In [ ]:
# Avaliação do modelo
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Acurácia no conjunto de teste:', test_acc)

# Mapeamento das classes
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)

# Exemplo de uma previsão
img_path = 'imagens/test/A/1.png'  # Exemplo de caminho da imagem
img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Fazer a previsão
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

# Obter o nome da classe prevista
classes = list(class_indices.keys())
predicted_label = classes[predicted_class]
print("Predicted Label:", predicted_label)


In [ ]:
# Função para exibir o resultado da classificação na câmera
def display_classification(frame, predicted_letter):
    # Adicionar o texto da classificação na imagem
    cv2.putText(frame, f'Classificação: {predicted_letter}', (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    
    # Exibir a imagem com a classificação
    cv2.imshow('Classificacao de Libras', frame)

# Função para pré-processamento da imagem
def preprocess_image(img):
    img = cv2.resize(img, (64, 64))
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32') / 255.0
    return img

# Função para exibir o resultado da classificação
def display_prediction(img, label):
    cv2.putText(img, label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow('Camera', img)

# Inicializar a câmera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar o frame da camera")
        break
    
    # Mostrar a câmera com um label inicial
    display_prediction(frame, '"Espaco" -> classificar | "Q" -> sair')

    # Verificar se a tecla "Espaço" foi pressionada
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        # Tirar uma foto (frame) e pré-processar
        photo = preprocess_image(frame)
        
        # Fazer a previsão usando o modelo
        prediction = model.predict(photo)
        predicted_class = np.argmax(prediction)
        
        # Mapear a classe prevista para a letra correspondente
        classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W']
        predicted_letter = classes[predicted_class]

        # Exibir o resultado da classificação na câmera usando o OpenCV
        display_classification(frame, predicted_letter)

    # Fechar a câmera ao pressionar a tecla "q"
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
